# What is Kubernetes?

Kubernetes coordinates a highly available cluster of computers that are connected to work as a single unit, and automates the distribution and scheduling of application containers across a cluster efficiently.

### Features
 - **Service discovery and load balancing:** TODO
 - **Storage orchestration:** TODO
 - **Automated rollouts and rollbacks:** TODO
 - **Automatic bin packing:** TODO
 - **Self-healing:** TODO
 - **Secret and configuration management:** TODO

### Limitations
 - TODO
 - TODO
 - TODO
 - TODO

# How does Kubernetes work?
 - **Pods:** TODO
 - **Services:** TODO
 - **Ingress controller:** TODO
 - **Volumes:** TODO
 - **Namespaces:** TODO
 - **Secrets and configuration:** TODO

# Let's Get Started!

We will be using a simple web app, created with [Flask](https://flask.palletsprojects.com/en/2.0.x/), to demonstrate the core functionality of Kubernetes. The `app/main.py` script contains code which configures and initializes a local Flask development server. Its contents are included below:

In [1]:
from flask import Flask
app = Flask(__name__)


@app.route("/")
def hello():
    return "Hello from Python/Flask!"


if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.1.104:5000/ (Press CTRL+C to quit)


Kubernetes allows you to deploy containerized applications to a cluster without tying them specifically to individual machines. A Kubernetes cluster is composed of two types of resources. First, the **Control Plane** handles the management of the cluster, coordinating all activities therein, such as: scheduling applications, maintaining applications' desired state, scaling applications, and rolling out application updates. Second, **Nodes** are physical computers or virtual machines that serve as worker machines in a cluster, running application instances. The following diagram illustrates the architecture of a Kubernetes cluster:

<div style="width: 100%;"><img src="img/module_01_cluster.svg"/></div>

The individual nodes in a cluster communicate with the control plane using the [Kubernetes API](https://kubernetes.io/docs/concepts/overview/kubernetes-api/). If you followed the setup steps for this workshop (located in `README.md`), and did not encounter any issues along the way, you should already have a cluster running locally. Run the following in your terminal/command prompt to view information about active clusters:

In [ ]:
kubectl cluster-info

Then, you can use the following command to view the names, statuses, roles, ages, and versions of individual Nodes:

In [ ]:
kubectl get nodes

Once you have a Kubernetes cluster running, you can begin to deploy your containerized application by creating a **Deployment** configuration. This Deployment tells Kubernetes how to create and update instances of your application. Once a Deployment has been created, the control plane schedules the application instances included in that Deployment to run on individual Nodes in the cluster. A **Deployment Controller** continuously monitors the created application instances, replacing instances that go down or are deleted with an instance running on another Node in the cluster. This provides a self-healing mechanism to address machine failure or maintenance. Now that we have a better idea of what goes on inside individual Nodes in a cluster, let's take a look at an updated cluster diagram.

<div style="width: 100%;"><img src="img/module_02_cluster.svg"/></div>

Before you deploy your Flask application, you need to containerize it. To do so, you must first navigate to the `app/` directory within this workshop folder in a terminal/command prompt window. Starting in the top-level repository directory `.../2022-spring-workshops`, you can use the following command to do so:

In [ ]:
cd "01 - Kubernetes"/app

You then need to point your shell to minikube's docker-daemon by first running the following command:

In [ ]:
minikube docker-env

Next, run the command from the last line of the output in your shell. For example, in the Window's command prompt, run:

In [ ]:
@FOR /f "tokens=*" %i IN ('minikube -p minikube docker-env') DO @%i

Then, you need to create an image of the application (think of this as a snapshot of the app's current state), which can be done with the `docker build` command as follows:

In [ ]:
docker build -f Dockerfile -t flask-app:1.0 .

The `-f` option allows you to specify the filename of the Dockerfile to build the container image from. Using `-t` enables you to name the image and optional tag it with a version in the form `name:tag`. Here, we name the image `flask-app` and tag it as version `1.0`. The `.` at the end of the command specifies the context from which to build the image from, the current working directory in this case. Once your image has been built, you can run the following command to list your Docker images:

In [ ]:
docker image ls

If you see an image with a repository of `flask-app` and a tag of `1.0`, you successfully containerized your Flask application and are ready to deploy it locally with Kubernetes! By default, Kubernetes will attempt to pull Docker images from *the public Docker registry* when creating a Deployment. To avoid this behavior, you need to run a local Docker registry to host your container images. You can start a local Docker registry with the following command:

In [ ]:
docker run -d -p 5000:5000 --restart=always --name registry registry:2

Next, you need to tag the image you created so that you can push it to the local Docker registry you created. This can be achieved with the following command:

In [ ]:
docker tag flask-app:1.0 localhost:5000/flask-app:1.0

Then, push your container image to your local Docker registry with the following command:

In [ ]:
docker push localhost:5000/flask-app:1.0

Finally, you can create a new Deployment from the containerized Flask application by running the following command:

In [ ]:
kubectl create deployment flask-app --image=localhost:5000/flask-app:1.0

Running the `create deployment` command causes the following actions to be performed:
- searching for a suitable node where an instance of the application could be run
- scheduling the application to run on that Node
- configuring the cluster to reschedule the instance on a new Node when needed

You can then list your Deployments with the following command:

In [ ]:
kubectl get deployments

The `READY` column indicates the number of instances of your app that are running, followed by the number of deployments you created. Your output should indicate that there is 1 instance of your app running on the 1 Deployment you created. Currently, your deployed application can only be accessed from inside Kubernetes/kubectl. Later on you will learn how to set up a **Service** to expose your application, but for now you will use a simple proxy server provided by `kubectl`. *Open a new terminal/command prompt window* and run the following command:

In [ ]:
kubectl proxy

You can then view all the APIs hosted through the proxy endpoint with the following command, or by copy-pasting the URL into your web browser:

In [ ]:
curl http://localhost:8001

Run `kubectl get pods` and note the name of the Pod running your `flask-app`. You can then access the pod directly through the API by running the following command, or by copy-pasting the URL into your web browser (make sure to replace `POD-NAME` with the name of the Pod you noted previously): 

In [ ]:
curl http://localhost:8001/api/v1/namespaces/default/pods/POD-NAME/

What is a Kubernetes **Pod**, though? Thus far, you may have a general idea of what Pods are, but we will now formally define them. Pods represent groups of one or more application containers (e.g., Docker containers), and some shared resources for those containers, including: shared storage (i.e., **Volumes**), networking (i.e., a unique cluster IP address), and information about how to run each individual container (e.g., container image version or specific ports to use). They are the atomic unit on the Kubernetes platform. When you create a Deployment, that Deployment will create Pod(s) with containers inside of them. The following infographic illustrates a few examples of Kubernetes Pods:

<div style="width: 100%;"><img src="img/module_03_pods.svg"/></div>

A **Pod** runs on a **Node**. A **Node** can have multiple **Pods**, and the **Control Plane** automatically handles scheduling the **Pods** across the **Nodes** in the cluster. Every Kubernetes Node runs at least:
- Kubelet, a process responsible for communication between the Kubernetes control plane and the Node; it manages the Pods and the containers running on a machine.
- A container runtime (like Docker) responsible for pulling the container image from a registry, unpacking the container, and running the application.

The architecture of a Kubernetes Node is illustrated by the following image:

<div style="width: 100%;"><img src="img/module_03_nodes.svg"/></div>

Up to this point, you have used a few of the commands implemented by the `kubectl` command-line interface. Some of the most well known `kubectl` operations are:
- **`kubectl get`** - list resources
- **`kubectl describe`** - show detailed information about a resource
- **`kubectl logs`** - print the logs from a container in a pod
- **`kubectl exec`** - execute a command on a container in a pod

You will now use some of the above commands to explore your existing application Deployment. Use the following command to see extensive details about the Pods in your cluster:

In [ ]:
kubectl describe pods

The `describe` command can be used to get detailed information about most of the Kubernetes primitives, such as Nodes, Pods, and Deployments. Similar to before, note the name of your Pod from the output of the `describe` command for use shortly. You can then access the Pod's API directly by running the following command, or by copy-pasting the URL into your web browser (make sure to replace POD-NAME with the name of the Pod you noted previously):

In [ ]:
curl http://localhost:8001/api/v1/namespaces/default/pods/POD-NAME/proxy/

Since your Flask application is incredibly simple, you should be greeted by a straightforward response from the Pod which contains the message: `Hello from Python!`. You can then look at the logs for the Pod you just sent an HTTP request to with the following command (make sure to replace POD-NAME with the name of the Pod you noted previously):

In [ ]:
kubectl logs POD-NAME

Anything that the application instance running in the Pod would regularly print to `STDOUT` (the standard output stream) is recorded in the logs for the container within the Pod. For your application, you should see some messages from when the Flask development server started up as well as one or more lines describing HTTP requests made to the server. Whether you used `curl` to send a request, or you navigated to the URL in your web browser, you should see at least one HTTP GET request listed in the output from `kubectl logs`.

Another useful command to know is `kubectl exec`, which allows you to execute commands directly on the container running in a Pod. The following command lists the environment variables for the container in the Pod that you have been working with thus far (make sure to replace POD-NAME with the name of the Pod you noted previously):

In [ ]:
kubectl exec POD-NAME -- env

To further illustrate the utility of `kubectl exec`, run the following command to start an interactive shell session in the Pod's container (make sure to replace POD-NAME with the name of the Pod you noted previously):

In [ ]:
kubectl exec -ti POD-NAME -- /bin/sh

Now that you are able to execute shell commands from within the container, run the following command to view the contents of the `main.py` script containing the code for the Flask application:

In [ ]:
cat main.py

To close your connection to the Pod's container, simply type `exit` and press your enter key.

Next, you will learn about Kubernetes **Services** and **Labels**. A Service routes traffic across a set of Pods, allowing Pods to die and replicate in Kubernetes without impacting your application. Labels are key/value pairs attached to objects and can be used in any number of ways:
- Designate objects for development, test, and production
- Embed version tags
- Classify an object using tags

Labels can be attached to objects at creation time or later on, and can be modified at any time. The following image illustrates the architecture of a cluster which utilizes services and labels:

<div style="width: 100%;"><img src="img/module_04_labels.svg"/></div>

To create a new Service and expose it to external traffic, we'll use the `kubectl expose` command, as follows:

In [ ]:
kubectl expose deployment/flask-app --type=LoadBalancer --port=8080 --target-port=80

Notice here that you expose port 8080 externally (with the `--port` option), and direct traffic to port 80 of the container itself (with the `--target-port` option). In essence, you can now access the container via port 8080 of the external IP address of the newly created Service. What is the external IP address of the new Service, though? You can use the `kubectl get` command to view some details about active Services in your cluster:

In [ ]:
kubectl get services

You should see two Services listed: one named `kubernetes` (which defines the cluster's IP address), and one named `flask-app` which you just created. Note the `EXTERNAL-IP` of the `flask-app` Service you created, which will likely be `localhost`. Next, use the following command to display detailed information about the newly created Service:

In [ ]:
kubectl describe services/flask-app

The Service you created allows you to access the Flask application directly, without routing your request through the Kubernetes proxy server that you should still have running. To do so, run the following command (make sure to replace `EXTERNAL-IP` with the `EXTERNAL-IP` listed in the `kubectl get services` output from before):

In [ ]:
curl EXTERNAL-IP:8080

Upon running this command, you should see the same, simple message returned from the Flask application as before. You can now close the separate terminal/command prompt window which you had running `kubectl proxy`.

When you created your Deployment, a label was automatically created for your Pod. Run the following command to view the name of the label created:

In [ ]:
kubectl describe deployment

The label for your Pod should be `app=flask-app`. Now, you can run the `kubectl get` command with the `-l` option which allows you to only get Pods with a label matching the one specified. Run the following:

In [ ]:
kubectl get pods -l app=flask-app

You likely only have one Pod in your cluster, so the output of the above command will likely be identical to the output of `kubectl get pods` without the `-l` option, but the ability to filter by label would be extremely useful in the case of multiple Pods/Deployments/etc. This option is perhaps better illustrated in such a case. Recall that the output of `kubectl get services` contains entries for a default Service created by Kubernetes, as well as the LoadBalancer Service that you created. Now try adding the `-l` option to `kubectl get services` as follows:

In [ ]:
kubectl get services -l app=flask-app

There should now only be the single Service that you created listed in the output. You can also give Labels to objects in your cluster manually, of course! Look back at the output from `kubectl get pods -l app=flask-app` and note the `NAME` of your Pod. Then, run the following command to add a new label to the Pod (make sure to replace POD-NAME with the name of the Pod):

In [ ]:
kubectl label pods POD-NAME version=v1

Then, use the following command to view the Pod's labels, among other information (make sure to replace POD-NAME with the name of the Pod you noted previously):

In [ ]:
kubectl describe pods POD-NAME

You should now see that the Pod has 3 labels: `app`, `pod-template-hash`, and `version`. Like before, we can filter the results of `kubectl get pods` with our newly created label with the following command:

In [ ]:
kubectl get pods -l version=v1